In [1]:
import pandas as pd
import numpy as np 
from Tech_Index import *

In [2]:
# Load data
price = pd.read_csv('../Data/ActiveSoybeanContractsforMarch2020.csv', delimiter='\t')
# price = pd.read_csv('../Data/ActiveSoybeanContractsforMay2020.csv', delimiter='\t')
# price = pd.read_csv('../Data/ActiveSoybeanContractsforJuly2020.csv', delimiter='\t')
soydata = pd.read_csv('../Data/soydata.csv', header=None)
news_count = pd.read_csv('../Data/News_count.csv')
twitter_count = pd.read_csv('../Data/Twitter_count.csv')
sp_gsci = pd.read_csv('../Data/SP_GSCI.csv')
export_index = pd.read_csv('../Data/ExportIndex_monthly.csv')

In [3]:
#  Add covariates calculated from Y 
price = tech_financial(price)

In [4]:
# Y
print(len(price.Close))
# impute price for missing dates with NA
idx = pd.date_range('11-14-2017', '08/30/2019')
price = price.set_index('Date')
price.index = pd.DatetimeIndex(price.index)
price = price.reindex(idx, fill_value='NaN')
print(len(price.Close))

452
655


In [413]:
# X_y: covariates calculated from Y 
X_y = price[price.columns.values[4:]].reset_index()
X_y = X_y.drop('index', axis=1)

In [414]:
# X1, X2 - news_count, twitter_count
print(len(news_count.Count)) # 11/14/2017 - 08/30/2019
print(len(twitter_count.Count))

655
655


In [415]:
# X3 - sp_gsci
# impute index for missing dates with NA
idx = pd.date_range('11/14/2017', '08/30/2019')
sp_gsci = sp_gsci.set_index('Date ')
sp_gsci.index = pd.DatetimeIndex(sp_gsci.index)
sp_gsci = sp_gsci.reindex(idx, fill_value='NaN')
len(sp_gsci.Agriculture)

655

In [416]:
# X4 - export_index
# impute index for missing dates with month values
# 2019/01 missing value for the whole month
idx = pd.date_range('11/01/2017', '08/30/2019')
export_index = export_index.set_index('DATE')
export_index.index = pd.DatetimeIndex(export_index.index)
export_index = export_index.reindex(idx, method='ffill')
export_index = export_index.loc['20171114':]
len(export_index.IQ00100)

655

In [417]:
# X5 # 11/14/2017 - 06/30/2019 (total 594), the rest is NA
soydata.shape
#len(pd.date_range('11/14/2017', '06/30/2019'))
#len(pd.date_range('11/14/2017', '07/31/2019'))
#soydata.iloc[594, ] 
#soydata.iloc[624, ] 

(655, 19)

In [418]:
soydata.iloc[594, ] 

0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
5        NaN
6        NaN
7        NaN
8        NaN
9      27.68
10     37.50
11     52.50
12     34.56
13     66.13
14     26.75
15    310.77
16    218.00
17    142.00
18    232.50
Name: 594, dtype: float64

In [235]:
soydata.iloc[500, ] 
soydata.iloc[654-(31*4), ] 

0     4607.243607
1     4222.070607
2     4251.055621
3     1993.449621
4        8.280000
5       17.600000
6       15.800000
7       20.500000
8        9.930000
9       27.860000
10      37.130000
11      54.000000
12      35.440000
13      65.250000
14      27.940000
15     304.260000
16     216.500000
17     166.500000
18     241.500000
Name: 530, dtype: float64

In [339]:
soydata.columns.values

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18])

In [250]:
# Combine all columns into dataframe 

In [419]:
data = pd.DataFrame({'date': pd.date_range('11-14-2017', '08/30/2019')})
data['price'] = price.Close.values
data[['EMA', 'MOM', 'ROC', '12day MA', '26day MA', 'Std_dev', 'Copp',
       'B1', 'B2', 'Trix', 'MACD', 'MACDsign', 'MACDdiff']] = X_y
data['news_count'] = news_count.Count
data['twitter_count'] = twitter_count.Count
data['sp_gsci'] = sp_gsci.Agriculture.values
data['export_index'] = export_index.IQ00100.values

In [12]:
data = data.replace('NaN', np.NaN)
data.fillna(method='ffill', inplace=True)

In [13]:
data[soydata.columns.values] = soydata # not fill na for soydata

In [422]:
data.columns.values

array(['date', 'price', 'EMA', 'MOM', 'ROC', '12day MA', '26day MA',
       'Std_dev', 'Copp', 'B1', 'B2', 'Trix', 'MACD', 'MACDsign',
       'MACDdiff', 'news_count', 'twitter_count', 'sp_gsci',
       'export_index', 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14,
       15, 16, 17, 18], dtype=object)

In [423]:
# Add prediction period data frame
idx = pd.date_range('11/14/2017', '11/08/2019')
# idx = pd.date_range('11/14/2017', '01/31/2020')
data = data.set_index('date')
data.index = pd.DatetimeIndex(data.index)
data = data.reindex(idx)
#export_index = export_index.loc['20171114':]
#len(export_index.IQ00100)

In [424]:
data.loc['2018-04-06',]

price             995.500000
EMA               998.841885
MOM                -8.500000
ROC                -0.007478
12day MA          999.854167
26day MA         1003.740385
Std_dev             5.029289
Copp               -0.015219
B1                  0.021875
B2                  0.333195
Trix               -0.000329
MACD               -2.185437
MACDsign           -1.089102
MACDdiff           -1.096335
news_count          0.000000
twitter_count       4.000000
sp_gsci           299.780000
export_index      201.400000
0                4661.985416
1                4209.859416
2                4438.092656
3                1749.316656
4                   9.850000
5                  18.000000
6                  17.500000
7                  23.300000
8                   9.920000
9                  29.700000
10                 31.190000
11                 54.000000
12                 37.310000
13                 66.880000
14                 29.500000
15                385.840000
16            

## Data Frame Generater

In [425]:
# lag_Y: depend Y_t on Y_(t-1), Y_(t-2),..., Y_(t-lag_Y)
# lag_0 = 31*4, lag_X: depend Y_t on X_(t-lag_0), X_(t-lag_0-1),..., X_(t-lag_0-lag_X)
# for monthly-valued columns in X, ignore lag_X
# example: lag_0 = 31*4+8, log_X = 2, lag_Y = 3 yield each row of form 
# Yt; Yt-1, Yt-2, Yt-3, X[set1]_{t-124}, X[set1]_{t-125}, X[set1]_{t-126}, X[set2]_{t-124}
# set1: news_counts, twitter_counts, sp_gsci 
# set2: export_index, soybean (monthly values)
def df_generater(data, lag_0, lag_X, lag_Y):
    n = data.shape[0]
    start_id = lag_0 + lag_X  
    data_new = data.copy()
    data_new = data_new[start_id:]
    data_new = data_new.reset_index()
    if lag_Y != 0:
        for i in range(lag_Y):
            name = 'Y_' + str(i+1)
            data_new[name] = data['price'][(start_id-i-1):(n-i-1)].values
        data_new = data_new.drop(['EMA', 'MOM', 'ROC', '12day MA', '26day MA', 'Std_dev', 'Copp',
       'B1', 'B2', 'Trix', 'MACD', 'MACDsign', 'MACDdiff'], axis=1)
    if lag_X != 0:
        # for X in set1 with daily values
        X_names = ['news_count', 'twitter_count', 'sp_gsci']
        data_new[['news_count_0', 
                  'twitter_count_0', 'sp_gsci_0']] = pd.DataFrame(data[X_names][(start_id-lag_0):(n-lag_0)].values)
        for i in range(lag_X):
            X_names_new = ['news_count', 'twitter_count', 'sp_gsci']
            for j in range(len(X_names_new)):
                X_names_new[j] = X_names_new[j] + '_' + str(i+1)
            dataframe = pd.DataFrame(data[X_names][(start_id-lag_0-i-1):(n-lag_0-i-1)].values)
            data_new[X_names_new] = dataframe
        data_new = data_new.drop(X_names, axis=1)
        
        # for X in set2 with monthly values
        X_names = ['export_index', 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
        X_names_new = ['export_index_0', '0_0', '1_0', '2_0', '3_0', '4_0', 
                   '5_0', '6_0', '7_0', '8_0', '9_0', '10_0', '11_0', '12_0', '13_0', '14_0', '15_0', 
                   '16_0', '17_0', '18_0']
        data_new[X_names_new] = pd.DataFrame(data[X_names][(start_id-lag_0):(n-lag_0)].values)
        data_new = data_new.drop(X_names, axis=1)
    data_new.rename(columns={'index': 'date'}, inplace=True)
    #data_new = data_new.drop('index', axis=1)
    print(data_new.shape)
    print(data_new.columns.values)
    return data_new

In [426]:
data_output = df_generater(data, lag_0=132, lag_X=10, lag_Y=0)

(583, 68)
['date' 'price' 'EMA' 'MOM' 'ROC' '12day MA' '26day MA' 'Std_dev' 'Copp'
 'B1' 'B2' 'Trix' 'MACD' 'MACDsign' 'MACDdiff' 'news_count_0'
 'twitter_count_0' 'sp_gsci_0' 'news_count_1' 'twitter_count_1'
 'sp_gsci_1' 'news_count_2' 'twitter_count_2' 'sp_gsci_2' 'news_count_3'
 'twitter_count_3' 'sp_gsci_3' 'news_count_4' 'twitter_count_4'
 'sp_gsci_4' 'news_count_5' 'twitter_count_5' 'sp_gsci_5' 'news_count_6'
 'twitter_count_6' 'sp_gsci_6' 'news_count_7' 'twitter_count_7'
 'sp_gsci_7' 'news_count_8' 'twitter_count_8' 'sp_gsci_8' 'news_count_9'
 'twitter_count_9' 'sp_gsci_9' 'news_count_10' 'twitter_count_10'
 'sp_gsci_10' 'export_index_0' '0_0' '1_0' '2_0' '3_0' '4_0' '5_0' '6_0'
 '7_0' '8_0' '9_0' '10_0' '11_0' '12_0' '13_0' '14_0' '15_0' '16_0' '17_0'
 '18_0']


In [427]:
data_output

,date,price,EMA,MOM,ROC,12day MA,26day MA,Std_dev,Copp,B1,...,9_0,10_0,11_0,12_0,13_0,14_0,15_0,16_0,17_0,18_0
0,2018-04-05,996.00,999.449501,-6.25,-0.007968,1000.375000,1004.134615,7.674226,-0.014772,0.022001,...,33.43,37.00,55.5,39.69,65.0,34.46,313.52,228.75,165.00,205.00
1,2018-04-06,995.50,998.841885,-8.50,-0.007478,999.854167,1003.740385,5.029289,-0.015219,0.021875,...,33.43,37.00,55.5,39.69,65.0,34.46,313.52,228.75,165.00,205.00
2,2018-04-07,995.50,998.841885,-8.50,-0.007478,999.854167,1003.740385,5.029289,-0.015219,0.021875,...,33.43,37.00,55.5,39.69,65.0,34.46,313.52,228.75,165.00,205.00
3,2018-04-08,995.50,998.841885,-8.50,-0.007478,999.854167,1003.740385,5.029289,-0.015219,0.021875,...,33.43,37.00,55.5,39.69,65.0,34.46,313.52,228.75,165.00,205.00
4,2018-04-09,998.25,998.750826,-4.75,-0.004488,999.145833,1003.355769,4.580529,-0.015396,0.021347,...,33.43,37.00,55.5,39.69,65.0,34.46,313.52,228.75,165.00,205.00
5,2018-04-10,1003.75,999.519930,1.00,0.001997,998.750000,1003.067308,5.496021,-0.012643,0.021647,...,33.43,37.00,55.5,39.69,65.0,34.46,313.52,228.75,165.00,205.00
6,2018-04-11,1002.50,999.978402,0.75,0.001248,998.833333,1003.182692,3.755829,-0.010813,0.021810,...,33.43,37.00,55.5,39.69,65.0,34.46,313.52,228.75,165.00,205.00
7,2018-04-12,1000.00,999.981725,-1.25,0.003261,998.895833,1003.096154,3.302461,-0.009802,0.021663,...,32.27,34.25,54.8,38.65,65.2,33.96,319.22,232.50,185.00,209.17
8,2018-04-13,997.50,999.599921,0.75,0.004279,998.791667,1002.625000,2.684446,-0.008983,0.021576,...,32.27,34.25,54.8,38.65,65.2,33.96,319.22,232.50,185.00,209.17
9,2018-04-14,997.50,999.599921,0.75,0.004279,998.791667,1002.625000,2.684446,-0.008983,0.021576,...,32.27,34.25,54.8,38.65,65.2,33.96,319.22,232.50,185.00,209.17


In [428]:
cutoff = data_output.index[data_output.date == '2019-08-30'].tolist()[0]
cutoff

512

In [429]:
data_train = data_output[0:(cutoff+1)]

In [430]:
data_train

,date,price,EMA,MOM,ROC,12day MA,26day MA,Std_dev,Copp,B1,...,9_0,10_0,11_0,12_0,13_0,14_0,15_0,16_0,17_0,18_0
0,2018-04-05,996.00,999.449501,-6.25,-0.007968,1000.375000,1004.134615,7.674226,-0.014772,0.022001,...,33.43,37.00,55.5,39.69,65.00,34.46,313.52,228.75,165.00,205.00
1,2018-04-06,995.50,998.841885,-8.50,-0.007478,999.854167,1003.740385,5.029289,-0.015219,0.021875,...,33.43,37.00,55.5,39.69,65.00,34.46,313.52,228.75,165.00,205.00
2,2018-04-07,995.50,998.841885,-8.50,-0.007478,999.854167,1003.740385,5.029289,-0.015219,0.021875,...,33.43,37.00,55.5,39.69,65.00,34.46,313.52,228.75,165.00,205.00
3,2018-04-08,995.50,998.841885,-8.50,-0.007478,999.854167,1003.740385,5.029289,-0.015219,0.021875,...,33.43,37.00,55.5,39.69,65.00,34.46,313.52,228.75,165.00,205.00
4,2018-04-09,998.25,998.750826,-4.75,-0.004488,999.145833,1003.355769,4.580529,-0.015396,0.021347,...,33.43,37.00,55.5,39.69,65.00,34.46,313.52,228.75,165.00,205.00
5,2018-04-10,1003.75,999.519930,1.00,0.001997,998.750000,1003.067308,5.496021,-0.012643,0.021647,...,33.43,37.00,55.5,39.69,65.00,34.46,313.52,228.75,165.00,205.00
6,2018-04-11,1002.50,999.978402,0.75,0.001248,998.833333,1003.182692,3.755829,-0.010813,0.021810,...,33.43,37.00,55.5,39.69,65.00,34.46,313.52,228.75,165.00,205.00
7,2018-04-12,1000.00,999.981725,-1.25,0.003261,998.895833,1003.096154,3.302461,-0.009802,0.021663,...,32.27,34.25,54.8,38.65,65.20,33.96,319.22,232.50,185.00,209.17
8,2018-04-13,997.50,999.599921,0.75,0.004279,998.791667,1002.625000,2.684446,-0.008983,0.021576,...,32.27,34.25,54.8,38.65,65.20,33.96,319.22,232.50,185.00,209.17
9,2018-04-14,997.50,999.599921,0.75,0.004279,998.791667,1002.625000,2.684446,-0.008983,0.021576,...,32.27,34.25,54.8,38.65,65.20,33.96,319.22,232.50,185.00,209.17


In [431]:
data_prediction = data_output[(cutoff+1):]

In [392]:
# data_prediction = data_output[(451+1):]

In [391]:
# data_output.index[data_output.date == '2019-06-30'].tolist()[0]

451

In [432]:
data_prediction

,date,price,EMA,MOM,ROC,12day MA,26day MA,Std_dev,Copp,B1,...,9_0,10_0,11_0,12_0,13_0,14_0,15_0,16_0,17_0,18_0
513,2019-08-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,27.86,37.13,54.0,35.44,65.25,27.94,304.26,216.50,166.50,241.50
514,2019-09-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,27.86,37.13,54.0,35.44,65.25,27.94,304.26,216.50,166.50,241.50
515,2019-09-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,27.86,37.13,54.0,35.44,65.25,27.94,304.26,216.50,166.50,241.50
516,2019-09-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,27.86,37.13,54.0,35.44,65.25,27.94,304.26,216.50,166.50,241.50
517,2019-09-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,27.86,37.13,54.0,35.44,65.25,27.94,304.26,216.50,166.50,241.50
518,2019-09-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,27.86,37.13,54.0,35.44,65.25,27.94,304.26,216.50,166.50,241.50
519,2019-09-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,27.86,37.13,54.0,35.44,65.25,27.94,304.26,216.50,166.50,241.50
520,2019-09-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,27.86,37.13,54.0,35.44,65.25,27.94,304.26,216.50,166.50,241.50
521,2019-09-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,27.86,37.13,54.0,35.44,65.25,27.94,304.26,216.50,166.50,241.50
522,2019-09-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,27.86,37.13,54.0,35.44,65.25,27.94,304.26,216.50,166.50,241.50


In [354]:
data_train.isnull().values.any()

False

In [355]:
data_train.isnull().sum()

date                0
price               0
EMA                 0
MOM                 0
ROC                 0
12day MA            0
26day MA            0
Std_dev             0
Copp                0
B1                  0
B2                  0
Trix                0
MACD                0
MACDsign            0
MACDdiff            0
news_count_0        0
twitter_count_0     0
sp_gsci_0           0
news_count_1        0
twitter_count_1     0
sp_gsci_1           0
news_count_2        0
twitter_count_2     0
sp_gsci_2           0
news_count_3        0
twitter_count_3     0
sp_gsci_3           0
news_count_4        0
twitter_count_4     0
sp_gsci_4           0
                   ..
sp_gsci_7           0
news_count_8        0
twitter_count_8     0
sp_gsci_8           0
news_count_9        0
twitter_count_9     0
sp_gsci_9           0
news_count_10       0
twitter_count_10    0
sp_gsci_10          0
export_index_0      0
0_0                 0
1_0                 0
2_0                 0
3_0       

In [356]:
# save to csv
data_train.to_csv('../Data/Train_July_X10_Y0_v1.csv')
data_prediction.to_csv('../Data/Prediction_July_X10_Y0_v1.csv')

In [394]:
# data_prediction.to_csv('../Data/Prediction_long.csv')

In [211]:
# def df_generater(data, lag_Y, lag_X):
#     n = data.shape[0]
#     m = max(lag_X, lag_Y)
#     data_new = data.copy()
#     data_new = data_new[m:]
#     data_new = data_new.reset_index()
#     if lag_Y != 0:
#         for i in range(lag_Y):
#             name = 'Y_' + str(i+1)
#             data_new[name] = data['price'][(m-i-1):(n-i-1)].values
#     if lag_X != 0:
#         X_names = data.columns.values[2:]
#         for i in range(lag_X):
#             X_names_new = X_names.copy()
#             for j in range(len(X_names_new)):
#                 X_names_new[j] = X_names_new[j] + '_' + str(i+1)
#             dataframe = pd.DataFrame(data[X_names][(m-i-1):(n-i-1)].values)
#             data_new[X_names_new] = dataframe
#     data_new = data_new.drop('index', axis=1)
#     print(data_new.shape)
#     return data_new

# Qiuyun

In [18]:
# Load data
price = pd.read_csv('../Data/ActiveSoybeanContractsforMarch2020.csv', delimiter='\t')
# price = pd.read_csv('../Data/ActiveSoybeanContractsforMay2020.csv', delimiter='\t')
# price = pd.read_csv('../Data/ActiveSoybeanContractsforJuly2020.csv', delimiter='\t')
soydata = pd.read_csv('../Data/soydata_new.csv', header=None)
news_count = pd.read_csv('../Data/News_count.csv')
twitter_count = pd.read_csv('../Data/Twitter_count.csv')
sp_gsci = pd.read_csv('../Data/SP_GSCI.csv')
export_index = pd.read_csv('../Data/ExportIndex_monthly.csv')

# Y
print(len(price.Close))
# impute price for missing dates with NA
idx = pd.date_range('11-14-2017', '08/30/2019')
price = price.set_index('Date')
price.index = pd.DatetimeIndex(price.index)
price = price.reindex(idx, fill_value='NaN')
print(len(price.Close))

# X1, X2 - news_count, twitter_count
print(len(news_count.Count)) # 11/14/2017 - 08/30/2019
print(len(twitter_count.Count))

# X3 - sp_gsci
# impute index for missing dates with NA
idx = pd.date_range('11/14/2017', '08/30/2019')
sp_gsci = sp_gsci.set_index('Date ')
sp_gsci.index = pd.DatetimeIndex(sp_gsci.index)
sp_gsci = sp_gsci.reindex(idx, fill_value='NaN')
len(sp_gsci.Agriculture)


# X4 - export_index
# impute index for missing dates with month values
# 2019/01 missing value for the whole month
idx = pd.date_range('11/01/2017', '08/30/2019')
export_index = export_index.set_index('DATE')
export_index.index = pd.DatetimeIndex(export_index.index)
export_index = export_index.reindex(idx, method='ffill')
export_index = export_index.loc['20171114':]
len(export_index.IQ00100)

data = pd.DataFrame({'date': pd.date_range('11-14-2017', '08/30/2019')})
data['price'] = price.Close.values
data['news_count'] = news_count.Count
data['twitter_count'] = twitter_count.Count
data['sp_gsci'] = sp_gsci.Agriculture.values
data['export_index'] = export_index.IQ00100.values

data = data.replace('NaN', np.NaN)
data.fillna(method='ffill', inplace=True)

data[soydata.columns.values] = soydata # not fill na for soydata

452
655
655
655


In [19]:
data

,date,price,news_count,twitter_count,sp_gsci,export_index,0,1,2,3,...,9,10,11,12,13,14,15,16,17,18
0,2017-11-14,978.50,0.0,0,282.85,187.4,4553.964210,4160.661210,3760.024669,2133.801669,...,33.43,37.0,55.5,39.69,65.00,34.46,313.52,228.75,165.0,205.0
1,2017-11-15,985.00,0.0,2,282.12,187.4,4553.964210,4160.661210,3760.024669,2133.801669,...,33.43,37.0,55.5,39.69,65.00,34.46,313.52,228.75,165.0,205.0
2,2017-11-16,983.00,0.0,4,281.98,187.4,4553.964210,4160.661210,3760.024669,2133.801669,...,33.43,37.0,55.5,39.69,65.00,34.46,313.52,228.75,165.0,205.0
3,2017-11-17,996.50,0.0,0,285.16,187.4,4553.964210,4160.661210,3760.024669,2133.801669,...,33.43,37.0,55.5,39.69,65.00,34.46,313.52,228.75,165.0,205.0
4,2017-11-18,996.50,0.0,0,285.16,187.4,4553.964210,4160.661210,3760.024669,2133.801669,...,33.43,37.0,55.5,39.69,65.00,34.46,313.52,228.75,165.0,205.0
5,2017-11-19,996.50,0.0,4,285.16,187.4,4553.964210,4160.661210,3760.024669,2133.801669,...,33.43,37.0,55.5,39.69,65.00,34.46,313.52,228.75,165.0,205.0
6,2017-11-20,998.50,0.0,0,284.01,187.4,4553.964210,4160.661210,3760.024669,2133.801669,...,33.43,37.0,55.5,39.69,65.00,34.46,313.52,228.75,165.0,205.0
7,2017-11-21,998.00,0.0,0,284.06,187.4,4553.964210,4160.661210,3760.024669,2133.801669,...,33.43,37.0,55.5,39.69,65.00,34.46,313.52,228.75,165.0,205.0
8,2017-11-22,1003.00,0.0,4,286.00,187.4,4553.964210,4160.661210,3760.024669,2133.801669,...,33.43,37.0,55.5,39.69,65.00,34.46,313.52,228.75,165.0,205.0
9,2017-11-23,1003.00,0.0,0,286.00,187.4,4553.964210,4160.661210,3760.024669,2133.801669,...,33.43,37.0,55.5,39.69,65.00,34.46,313.52,228.75,165.0,205.0


In [20]:
data.to_csv('../Data/zhu.csv')